In [148]:
# iPyTest allows us to solve AoC using test-driven development principals.
import ipytest
ipytest.autoconfig(addopts=['--color=no'])

In [3]:
# Modules to support development
import os
import re
import collections
import itertools
import functools
import logging
import pprint
import numpy as np

In [229]:
%%ipytest

class Monkey():
    def __init__(self, _id, items, operation_text, test, throws, divisor, part2mode=False):
        self.id = _id
        self.items = []
        self.operation_text = operation_text
        self.operation = lambda x: x
        if operation_text:
            self.operation = lambda old: eval(operation_text, {"old": old}, {})
        self.test = test
        self.throws = throws
        self.inspected = 0
        self.divisor = divisor
        if self.divisor == None:
            self.divisor = 1
        self.part2mode = part2mode
        self.gcd = 1

        for item in items:
            #self.items.append(self.operation(item))
            self.receive(item)

    def __str__(self):
        return f"Monkey {self.id} ({self.inspected}): {self.items}"
        
    def receive(self, worry_level):
        self.items.append(worry_level)
    
    def throw(self, monkeys, ):
        while len(self.items) > 0:
            item = self.items.pop(0) # pop zero isn't most efficent
            self.inspected += 1

            if self.part2mode == False:
                worry_level = self.operation(item) // 3
            else:
                worry_level = self.operation(item) % self.gcd

            if self.test(worry_level):
                monkeys[self.throws[0]].receive(worry_level)
            else:
                monkeys[self.throws[1]].receive(worry_level)

def parse_monkey(monkey_lines, part2mode=False):
    mm = re.match("Monkey (\d+)\:", monkey_lines[0])
    if not mm:
        raise ValueError(monkey_lines[0])

    _id = int(mm.group(1))

    ii = re.match("\s+Starting items: (.*)$", monkey_lines[1])
    if not ii:
        raise ValueError()

    items = [ int(xx) for xx in ii.group(1).split(",") ]

    oo = re.match("\s+Operation: new = (.*)$", monkey_lines[2])
    if not oo:
        raise ValueError

    operation_text = oo.group(1)

    oo = re.match("\s+Operation: new = (.*)$", monkey_lines[2])
    if not oo:
        raise ValueError

    tt = re.match("\s+Test: divisible by (\d+)$", monkey_lines[3])
    if not tt:
        raise ValueError

    divisor = int(tt.group(1))
    test = lambda xx: ( xx % divisor) == 0
    
    xx = re.match("\s+If true: throw to monkey (\d+)$", monkey_lines[4])
    if not xx:
        raise ValueError

    yy = re.match("\s+If false: throw to monkey (\d+)$", monkey_lines[5])
    if not yy:
        raise ValueError
    
    throws = (int(xx.group(1)), int(yy.group(1)))

    monkey = Monkey(
        _id,
        items,
        operation_text,
        test,
        throws,
        divisor,
        part2mode
    )

    monkey.operation_text = operation_text

    return monkey
    
def test_parse_monkey():
    monkey_input = [
        "Monkey 0:",
        "  Starting items: 79, 98",
        "  Operation: new = old * 19",
        "  Test: divisible by 23",
        "    If true: throw to monkey 2",
        "    If false: throw to monkey 3",
    ]
    
    monkey = parse_monkey(monkey_input)
    assert monkey.id == 0
    assert monkey.items == [79, 98]
    assert monkey.operation(old=5) == 5 * 19
    assert monkey.operation(old=8) == 8 * 19
    assert monkey.test(33) == False
    assert monkey.test(46) == True
    assert monkey.throws[0] == 2
    assert monkey.throws[1] == 3

    monkeys = [ 
        Monkey(None, [], None, None, None, None),
        Monkey(None, [], None, None, None, None),
        Monkey(None, [], None, None, None, None),
        Monkey(None, [], None, None, None, None),
    ]

    monkey.throw(monkeys)

    assert monkey.items == [ ]
    assert monkeys[0].items == [ ]
    assert monkeys[1].items == [ ]
    assert monkeys[2].items == [ ]
    assert monkeys[3].items == [ 500, 620]
    assert monkey.inspected == 2
    
    monkey_input = [
        "Monkey 1:",
        "  Starting items: 54, 65, 75, 74",
        "  Operation: new = old + 6",
        "  Test: divisible by 19",
        "    If true: throw to monkey 2",
        "    If false: throw to monkey 0",
    ]
    
    monkey = parse_monkey(monkey_input)
    assert monkey.id == 1
    assert monkey.items == [ 54, 65, 75, 74 ]
    assert monkey.operation(old=5) == 5 + 6
    assert monkey.operation(old=8) == 8 + 6
    assert monkey.test(33) == False
    assert monkey.test(38) == True
    assert monkey.throws[0] == 2
    assert monkey.throws[1] == 0

    monkeys = [ 
        Monkey(None, [], None, None, None, None),
        Monkey(None, [], None, None, None, None),
        Monkey(None, [], None, None, None, None),
        Monkey(None, [], None, None, None, None),
    ]

    monkey.throw(monkeys)

    assert monkey.items == [ ]
    assert monkeys[0].items == [ 20, 23, 27, 26]
    assert monkeys[1].items == [ ]
    assert monkeys[2].items == [ ]
    assert monkeys[3].items == [ ]
    assert monkey.inspected == 4

======================================= test session starts =======================================
platform win32 -- Python 3.10.9, pytest-7.2.0, pluggy-1.0.0
rootdir: c:\Users\MichaelIhde\Git\advent-of-code-2022\python
collected 1 item

t_73c2c3b05c4e448189fcb5405e1e418e.py .                                                      [100%]

======================================== 1 passed in 0.01s ========================================


In [230]:
%%ipytest


def part1(puzzle_input):
    if not os.path.exists(puzzle_input):
        return

    with open(puzzle_input) as ff:
        dd = ff.readlines()

    monkeys = {}
    ii = 0
    while ii + 6 <= len(dd):
        monkey = parse_monkey(dd[ii:ii+6])
        monkeys[monkey.id] = monkey # this code doesn't assume monkeys are in order in the input file
        ii += 7

    for rr in range(20):
        print(f"Round {rr+1}")
        for monkey_id, monkey in sorted(monkeys.items(), key=lambda x: x[0]):
            monkey.throw(monkeys)

        for monkey_id, monkey in sorted(monkeys.items(), key=lambda x: x[0]):
            print(monkey) 

    most_active = []
    for monkey_id, monkey in sorted(monkeys.items(), key=lambda x: x[1].inspected, reverse=True):
        print(f"Monkey {monkey.id} inspected items {monkey.inspected} times.")
        if len(most_active) < 2:
            most_active.append(monkey.inspected)

    return most_active[0] * most_active[1]

def test_part1():
    assert part1(os.path.join("..", "dat", "day11_test.txt")) == 10605
    
    
part1(os.path.join("..", "dat", "day11.txt"))

Round 1
Monkey 0 (6): [26, 380, 525, 430, 405, 462, 196, 209]
Monkey 1 (4): []
Monkey 2 (3): [12, 10, 10, 11, 8, 239, 239, 263]
Monkey 3 (11): [33, 18, 18, 32, 26, 18, 132, 84, 76, 138, 79, 1046]
Monkey 4 (7): [26, 13, 13]
Monkey 5 (14): []
Monkey 6 (11): [28, 25, 11, 7, 33]
Monkey 7 (5): []
Round 2
Monkey 0 (14): [6, 6, 82, 82, 90, 164, 82, 82, 31, 31, 31]
Monkey 1 (7): []
Monkey 2 (11): [11, 10, 5, 4, 12, 5, 4, 4, 5, 5, 4, 16, 11, 10, 17, 10, 118]
Monkey 3 (23): [549, 622, 586, 284, 302, 1394, 445334]
Monkey 4 (13): []
Monkey 5 (22): []
Monkey 6 (28): [191690]
Monkey 7 (6): []
Round 3
Monkey 0 (25): [6, 4, 6, 4, 4, 4, 6, 8, 31, 19, 19, 19, 19, 44, 31, 31, 259]
Monkey 1 (7): []
Monkey 2 (28): [63898, 63, 71, 67, 33, 35, 157, 49483]
Monkey 3 (30): [9, 9, 119, 119, 131, 237, 119, 119]
Monkey 4 (22): []
Monkey 5 (33): []
Monkey 6 (36): [15, 15, 15]
Monkey 7 (9): []
Round 4
Monkey 0 (42): [26, 24, 14, 54, 16496, 134906, 145, 82]
Monkey 1 (7): []
Monkey 2 (36): [6, 6, 6, 3, 3, 15, 15, 16, 

57348

======================================= test session starts =======================================
platform win32 -- Python 3.10.9, pytest-7.2.0, pluggy-1.0.0
rootdir: c:\Users\MichaelIhde\Git\advent-of-code-2022\python
collected 1 item

t_73c2c3b05c4e448189fcb5405e1e418e.py .                                                      [100%]

======================================== 1 passed in 0.03s ========================================


In [234]:
%%ipytest


def part2(puzzle_input):
    if not os.path.exists(puzzle_input):
        return

    with open(puzzle_input) as ff:
        dd = ff.readlines()

    monkeys = {}
    ii = 0
    while ii + 6 <= len(dd):
        monkey = parse_monkey(dd[ii:ii+6], part2mode=True)
        monkeys[monkey.id] = monkey # this code doesn't assume monkeys are in order in the input file
        ii += 7

    gcd = 1
    for monkey in monkeys.values():
        gcd *= monkey.divisor

    print("GCD", gcd)
    for monkey in monkeys.values():
        monkey.gcd = gcd
        
    for rr in range(10000):
        #print("-"*80)
        #print(f"Round {rr+1}")
        for monkey_id, monkey in sorted(monkeys.items(), key=lambda x: x[0]):
            monkey.throw(monkeys)

        #for monkey_id, monkey in sorted(monkeys.items(), key=lambda x: x[0]):
        #    print(monkey) 

    for monkey_id, monkey in monkeys.items():
        print(f"Monkey {monkey.id} inspected items {monkey.inspected} times.")

    most_active = []
    for monkey_id, monkey in sorted(monkeys.items(), key=lambda x: x[1].inspected, reverse=True):
        if len(most_active) < 2:
            most_active.append(monkey.inspected)

    return most_active[0] * most_active[1]

def test_part2():
    assert part2(os.path.join("..", "dat", "day11_test.txt")) == 2713310158
    
    
part2(os.path.join("..", "dat", "day11.txt"))

GCD 9699690
Monkey 0 inspected items 116421 times.
Monkey 1 inspected items 23734 times.
Monkey 2 inspected items 112859 times.
Monkey 3 inspected items 106904 times.
Monkey 4 inspected items 57020 times.
Monkey 5 inspected items 109357 times.
Monkey 6 inspected items 121166 times.
Monkey 7 inspected items 22573 times.


14106266886

======================================= test session starts =======================================
platform win32 -- Python 3.10.9, pytest-7.2.0, pluggy-1.0.0
rootdir: c:\Users\MichaelIhde\Git\advent-of-code-2022\python
collected 1 item

t_73c2c3b05c4e448189fcb5405e1e418e.py .                                                      [100%]

======================================== 1 passed in 0.99s ========================================
